In [1]:
import os
os.chdir("../")

import gzip
import pandas as pd
import numpy as np

import geopandas as gpd
import shapely
from shapely.geometry import mapping, MultiPolygon, Point, Polygon
import h3

In [2]:
filepath = os.getcwd() + "/data/685_buildings.csv"
data = pd.read_csv(filepath)
data["geometry"] = shapely.wkt.loads(data['geometry'])

In [3]:
gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

In [4]:
h3_level = 6
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.latitude, row.longitude, h3_level)

gdf['hex6'] = gdf.apply(lat_lng_to_h3, axis=1)

In [5]:
ng_hex = gdf.groupby("hex6")[["area_in_meters"]].sum().reset_index().drop_duplicates()
ng_hex["geometry"] = [Polygon(h3.h3_to_geo_boundary(i,geo_json=True)) for i in ng_hex.hex6]
ng_hex_gdf = gpd.GeoDataFrame(ng_hex, crs="EPSG:4326")
ng_hex_gdf.head(5)

,hex6,area_in_meters,geometry
0,8672d0427ffffff,24669.8240,"POLYGON ((140.85206 -4.45146, 140.84573 -4.486..."
1,8672d0487ffffff,2109.6011,"POLYGON ((140.81407 -4.66372, 140.80774 -4.699..."
2,8672d0497ffffff,7096.1550,"POLYGON ((140.77354 -4.71159, 140.76721 -4.746..."
3,8672d049fffffff,6755.5535,"POLYGON ((140.83562 -4.72189, 140.82929 -4.757..."
4,8672d04afffffff,8.7050,"POLYGON ((140.85461 -4.61583, 140.84828 -4.651..."


In [ ]:
ng_hex_gdf.explore()